In [2]:
import os
import pandas as pd
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import binom
from statsmodels.stats.multitest import fdrcorrection 

In [6]:
terms={
    "fall_2014" : pd.read_csv("/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocfall2014_ampadj.csv"),
    "spring_2015" : pd.read_csv("/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocspring2015_ampadj.csv"),
    "fall_2015" : pd.read_csv("/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocfall2015_ampadj.csv"),
    "spring_2016" : pd.read_csv("/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocspring2016_ampadj.csv")
}


In [8]:

TERMS = {
    "Fall 2014": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocfall2014_ampadj.csv",
    "Spring 2015": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocspring2015_ampadj.csv",
    "Fall 2015": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocfall2015_ampadj.csv",
    "Spring 2016": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocspring2016_ampadj.csv",
}

MIN_EVENTS = 2

# --- circular descriptives per student/term ---
def descriptives_one_term(path, term, min_events=MIN_EVENTS):
    df = pd.read_csv(path)
    df["alpha"] = df["rad"].astype(float) % (2*np.pi)
    keep = df.groupby("PSEUDO_ID").size().loc[lambda s: s >= min_events].index
    df = df[df["PSEUDO_ID"].isin(keep)]

    rows = []
    for sid, g in df.groupby("PSEUDO_ID"):
        a = (g["alpha"].dropna().to_numpy(float) % (2*np.pi))
        n = a.size
        if n < 2:
            rows.append(dict(
                TERM=term, PSEUDO_ID=sid, n_events=int(n),
                C=np.nan, S=np.nan, R_bar=np.nan,
                mu_rad=np.nan, mu_deg=np.nan, mu_hours=np.nan,
                sigma_rad=np.nan, sigma_deg=np.nan, sigma_hours=np.nan, two_x_sigma_hours=np.nan,
                TERM_GPA=g["TERM_GPA"].astype(float).dropna().mean() if "TERM_GPA" in g else np.nan,
                chronot=g["chronot"].iloc[0] if "chronot" in g.columns and len(g) else np.nan
            ))
            continue

        C = float(np.cos(a).sum()) #sum of cosines of all angles
        S = float(np.sin(a).sum()) #sum of sines of all angles
        R = np.hypot(C, S) #resultant vector length
        R_bar = float(R / n) #mean resultant vector length
        mu = float(np.arctan2(S, C) % (2*np.pi)) #mean direction
        sigma = float(np.sqrt(-2*np.log(R_bar))) if R_bar > 0 else np.nan #circular standard deviation

        rows.append(dict(
            TERM=term,
            PSEUDO_ID=sid,
            n_events=int(n),
            C=C, S=S, R_bar=R_bar,
            mu_rad=mu,
            mu_deg=float(np.degrees(mu)),
            mu_hours=float(mu * 24/(2*np.pi)),
            sigma_rad=sigma,
            sigma_deg=float(np.degrees(sigma)) if not np.isnan(sigma) else np.nan,
            sigma_hours=float(sigma * 24/(2*np.pi)) if not np.isnan(sigma) else np.nan,
            two_x_sigma_hours=float(2 * sigma * 24/(2*np.pi)) if not np.isnan(sigma) else np.nan,
            TERM_GPA=g["TERM_GPA"].astype(float).dropna().mean() if "TERM_GPA" in g else np.nan,
            chronot=g["chronot"].iloc[0] if "chronot" in g.columns and len(g) else np.nan
        ))
    return pd.DataFrame(rows)

#build the single output table
desc_all = pd.concat(
    [descriptives_one_term(p, t) for t, p in TERMS.items()],
    ignore_index=True
).sort_values(["TERM", "PSEUDO_ID"]).reset_index(drop=True)

# optional: save
# desc_all.to_csv("circular_descriptives_ALL_TERMS.csv", index=False)

desc_all.head()


,TERM,PSEUDO_ID,n_events,C,S,R_bar,mu_rad,mu_deg,mu_hours,sigma_rad,sigma_deg,sigma_hours,two_x_sigma_hours,TERM_GPA,chronot
0,Fall 2014,9187,16,-5.803632,-2.355448,0.391463,3.527138,202.090149,13.472677,1.369572,78.470717,5.231381,10.462762,4.0,Lark
1,Fall 2014,19491,33,-9.492917,-17.296234,0.597881,4.210435,241.240127,16.082675,1.014263,58.112967,3.874198,7.748396,4.0,Finch
2,Fall 2014,20089,3,-0.849697,-1.161348,0.479666,4.080738,233.809059,15.587271,1.212160,69.451631,4.630109,9.260217,3.0,NaN
3,Fall 2014,30559,85,-0.871804,-25.908922,0.304983,4.678753,268.072792,17.871519,1.541102,88.298654,5.886577,11.773154,4.0,Finch
4,Fall 2014,31539,22,-6.754000,-2.304582,0.324380,3.470422,198.840529,13.256035,1.500560,85.975742,5.731716,11.463432,4.0,Lark
